In [ ]:
import numpy as np
from datetime import datetime
import os
import itertools as it
from env import *
import pandas as pd
import random
from BR_agent import *
from MO_agent import *
import time



In [ ]:
e=tiger_POMDP_env(read_config=True,config_address='./tiger.json',parameters=None)
num_to_act=dict(zip(list(e.actions.values()),list(e.actions.keys())))


In [ ]:
dict(zip(list(e.actions.values()),list(e.actions.keys())))

### Check similarity of methods' actions

In [ ]:
E=[-1.5,-0.2,-0.1,0.1,0.2,1.5]
W=[-1,0,1,2]
inits=[[1,0],[0.9,0.1],[0.75,0.25],[0.5,0.5]]
initial_observation=0

loop_counts=0
for max_depth in range(1,3):
    corr_count=0
    all_count=0
    loop_counts+=1
    print(loop_counts)
    for e1 in E:
        for e2 in E:
            exp_vorfaktoren=[e1,e2]
            for w1 in W:
                for w2 in W:
                    exp_weights=[w1,w2]
                    #print(exp_vorfaktoren, exp_weights)
                    for init in inits:
                        ag2=Multi_Variate_agent(environment=e, planning_depth=max_depth, partitioning_chunk_number=10,agent_mode='cheating')
                        ag2.reset()
                        x_map2,M2,F2,G2,X2,value_function2,all_theta2=ag2.pre_planning(exp_vorfaktoren=exp_vorfaktoren,exp_weights=exp_weights, initial_theta=init,initial_observation=initial_observation)
                        v2,a2,q2,vf2=ag2.value_iteration()
                        
                        
                        ag=Bauerle_Rieder_agent(environment=e, num_of_Mu_chunks=2,max_iterations=max_depth)
                        b_S,b_r,b_map,b_q_func,b_value_function,b_action_func=ag.continious_optimized_planning(initial_mu_state=init,initial_observation=initial_observation,initial_wealth=0,exp_weights=exp_weights,exp_vorfaktoren=exp_vorfaktoren)
                        
                        #m_r,m_map, m_q_func,m_value_function,m_action_func=MO_cheating(env_dynamics=e,planning_depth=max_depth,exp_vorfaktoren=exp_vorfaktoren,exp_weights=exp_weights ,initial_theta=init,initial_observation=0,rounding_prec_coeff=10000)
                        #b_S,b_r,b_map,b_q_func,b_value_function,b_action_func=Beurele_cheating(env_dynamics=e,planning_depth=max_depth,exp_vorfaktoren=exp_vorfaktoren,exp_weights=exp_weights ,initial_mu_state=init,initial_wealth=0,comparison_precision=1.0e-5)
                        
                        mx_init=init*len(init)
                        mx_init.append(0)
                        mr_init=[0]*len(init)
                        m_init=(tuple(mx_init),tuple(mr_init),0)
                        b_init=(initial_observation,tuple(init),0)
                        all_count+=1
                        if a2[0][tuple(m_init)]==b_action_func[0][b_init]:
                            corr_count+=1
                        else:
                            #pass
                            print(q2[0][tuple(m_init)], b_q_func[0][tuple(init)],'           ',a2[0][tuple(m_init)],b_action_func[0][0][tuple(init)],'    ||     exp:',exp_vorfaktoren,exp_weights,init)
    print('--------------------------')                        
    print(all_count-corr_count,all_count,str(corr_count*100/all_count)+'%')
    print('=====================================')

## Simulation

### BR


In [ ]:
max_depth=4
planning_depth=max_depth
exp_vorfaktoren=[-1,0.1]
exp_weights=[-1,1]
initial_Mu=[0.5,0.5]
initial_observation=0
initial_wealth=0

In [ ]:
ta=time.time()
###############    Environment

e.reset()
initial_observation=e.current_state
observation=initial_observation
state=e.current_state
##############       Agent
# Create the agent
ag=Bauerle_Rieder_agent(environment=e, num_of_Mu_chunks=2,max_iterations=max_depth)


# Run its value iteration
b_S,b_reachables,b_map,b_q_func,b_value_function,b_action_func=ag.continious_optimized_planning(initial_mu_state=initial_Mu,initial_observation=initial_observation,initial_wealth=0,exp_weights=exp_weights,exp_vorfaktoren=exp_vorfaktoren)
statesused=0
for i in range(max_depth+1):
    statesused+=len(b_reachables[i])
print("depth:",max_depth, "states used:", statesused)
# reset the agent 
ag.ch_reset(initial_mu_state=initial_Mu,initial_observation=initial_observation,initial_wealth=initial_wealth)

tb=time.time()
print("time needed:", tb-ta)

#############         Simulation

for t in range(planning_depth):
    
    print('t=',t)
    print('------')
    print('state:',state,e.states[state])
    print('last_observation:',observation)
    print('-------------------------------')
    print('')
    print('current internal state:',ag.current_internal_state)
    print('')
    
    # agent select the action
    action,value_of_action,_=ag.ch_do_action()
    
    #environment feedback
    t1,t2,state,reward,observation=e.step(num_to_act[action])
    
    # agent update
    new_x=ag.ch_update_agent(new_observation=observation) 
    
    print('             action:',num_to_act[action])  
    print ('             **' )
    print('reward:',reward,'new observation:',observation)
    print('')
    print('new internal (Mu) state:', ag.current_internal_state)
    print('')
    print('============================')

### Multi Variate (Multi Objective)

In [ ]:
planning_depth=5
partitioning_chunk_number=1000
# exp_vorfaktoren=[-1,0.1]
# exp_weights=[-1,1]
exp_vorfaktoren=[1,-1]
exp_weights=[0.5,-0.5]
initial_theta=[0.5,0.5]
initial_observation=0
modes=['discrete_optimized','naive','cheating']

In [ ]:
###############    Environment
t0 = time.time()
e.reset()
initial_observation=e.current_state
observation=initial_observation
state=e.current_state

##############     Agent 

ag=Multi_Variate_agent(environment=e, planning_depth=planning_depth, partitioning_chunk_number=partitioning_chunk_number,agent_mode='cheating')
ag.reset()
mo_x_map,mo_M,mo_F,mo_G,mo_reachables,mo_value_function_zero,mo_all_thetas=ag.pre_planning(exp_vorfaktoren=exp_vorfaktoren,exp_weights=exp_weights, initial_theta=initial_theta,initial_observation=initial_observation,initial_wealth=0)
mo_value_func,mo_action_func,mo_q_func,vf2=ag.value_iteration()
t1 = time.time()
statesused=0
for i in range(planning_depth+1):
    statesused+=len(mo_reachables[i])
print("depth:",planning_depth, "states used:", statesused)
print("time needed:", t1-t0)

In [ ]:




##############       Simulation
for t in range(planning_depth):
    
    print('t=',t)
    print('------')
    print('state:',state,e.states[state])
    print('last_observation:',observation)
    print('-------------------------------')
    print('')
    print('current internal state:',ag.current_internal_state)
    print('')
    
    # agent select the action
    action,value_of_action=ag.do_action()
    
    #environment feedback
    t1,t2,state,reward,observation=e.step(num_to_act[action])
    
    # agent update
    new_x=ag.update_agent(new_observation=observation) 
    
    print('             action:',num_to_act[action])  
    print ('             **' )
    print('reward:',reward,'new observation:',observation)
    print('')
    print('new internal state:', ag.current_internal_state)
    print('')
    print('============================')

In [ ]:
mo_reachables[1]

In [ ]:
b_reachables[1]

In [ ]:
max_depth=3
planning_depth=max_depth
exp_vorfaktoren=[-1,0.1]
exp_weights=[-1,1]
initial_Mu=[0.5,0.5]
initial_observation=0
initial_wealth=0

In [ ]:
ta=time.time()
###############    Environment

e.reset()
initial_observation=e.current_state
observation=initial_observation
state=e.current_state
##############       Agent
# Create the agent
ag=Bauerle_Rieder_agent(environment=e, num_of_Mu_chunks=2,max_iterations=max_depth)


# Run its value iteration
b_S,b_reachables,b_map,b_q_func,b_value_function,b_action_func=ag.continious_optimized_planning(initial_mu_state=initial_Mu,initial_observation=initial_observation,initial_wealth=0,exp_weights=exp_weights,exp_vorfaktoren=exp_vorfaktoren)
statesused=0
for i in range(max_depth+1):
    statesused+=len(b_reachables[i])
print("depth:",max_depth, "states used:", statesused)
# reset the agent 
ag.ch_reset(initial_mu_state=initial_Mu,initial_observation=initial_observation,initial_wealth=initial_wealth)

tb=time.time()
print("time needed:", tb-ta)

#############         Simulation

for t in range(planning_depth):
    
    print('t=',t)
    print('------')
    print('state:',state,e.states[state])
    print('last_observation:',observation)
    print('-------------------------------')
    print('')
    print('current internal state:',ag.current_internal_state)
    print('')
    
    # agent select the action
    action,value_of_action,_=ag.ch_do_action()
    
    #environment feedback
    t1,t2,state,reward,observation=e.step(num_to_act[action])
    
    # agent update
    new_x=ag.ch_update_agent(new_observation=observation) 
    
    print('             action:',num_to_act[action])  
    print ('             **' )
    print('reward:',reward,'new observation:',observation)
    print('')
    print('new internal (Mu) state:', ag.current_internal_state)
    print('')
    print('============================')

In [ ]:
############################## MULTIVARIATE DATA GATHERING ################################################
tx=time.time()
loops=50

results={1:[],2:[],3:[],4:[],5:[],6:[],7:[],8:[],9:[],10:[]}
comptime={1:[],2:[],3:[],4:[],5:[],6:[],7:[],8:[],9:[],10:[]}
for i in range(1,7):
    
        for j in range(loops):

            planning_depth=i
            partitioning_chunk_number=1000
#           exp_vorfaktoren=[1,-1]
#           exp_weights=[0.5,-0.5]
            exp_vorfaktoren=[-1,0.1]
            exp_weights=[-1,1]
            initial_theta=[0.5,0.5]
            initial_observation=0
            modes=['discrete_optimized','naive','cheating']

            ###############    Environment
            t0 = time.time()
            e.reset()
            initial_observation=e.current_state
            observation=initial_observation
            state=e.current_state

            ##############     Agent 

            ag=Multi_Variate_agent(environment=e, planning_depth=planning_depth, partitioning_chunk_number=partitioning_chunk_number,agent_mode='cheating')
            ag.reset()
            mo_x_map,mo_M,mo_F,mo_G,mo_reachables,mo_value_function_zero,mo_all_thetas=ag.pre_planning(exp_vorfaktoren=exp_vorfaktoren,exp_weights=exp_weights, initial_theta=initial_theta,initial_observation=initial_observation,initial_wealth=0)
            mo_value_func,mo_action_func,mo_q_func,vf2=ag.value_iteration()
            t1 = time.time()
            statesused=0
            for i in range(planning_depth+1):
                statesused+=len(mo_reachables[i])
#             print("depth:",planning_depth, "states used:", statesused)
#             print("time needed:", t1-t0)
            comptime[i].append(t1-t0)

            ##############       Simulation
            total_reward=0
            for t in range(planning_depth):

#                 print('t=',t)
#                 print('------')
#                 print('state:',state,e.states[state])
#                 print('last_observation:',observation)
#                 print('-------------------------------')
#                 print('')
#                 print('current internal state:',ag.current_internal_state)
#                 print('')

                # agent select the action
                action,value_of_action=ag.do_action()
#                 if action=="open_right_low" or action=="open_left_low":
#                     low+=1
#                 elif action=="open_right_high" or action=="open_left_high":
#                     high+=1
#                 elif action=="listen"
#                     listen+=1

                #environment feedback
                t1,t2,state,reward,observation=e.step(num_to_act[action])
                total_reward+=reward

                # agent update
                new_x=ag.update_agent(new_observation=observation) 


#                 print('             action:',num_to_act[action])  
#                 print ('             **' )
#                 print('reward:',reward,'new observation:',observation)
#                 print('')
#                 print('new internal state:', ag.current_internal_state)
#                 print('')
#                 print('============================')
#                 print('total_reward', total_reward)

            results[i].append(total_reward)
        print(i)

print("results", results)
print("=================================")
print("time",comptime)
ty=time.time()
print("----------------------")
print("total time needed for", loops, "loops", ty-tx, "seconds")



In [ ]:
############################## BAUERLE DATA GATHERING ################################################


results_b={1:[],2:[],3:[]}
efficiency_b={1:[],2:[],3:[]}

for i in range(1,4):
    for j in range(2):
        total_reward=0
        max_depth=i
        planning_depth=max_depth
        exp_vorfaktoren=[1,-1]
        exp_weights=[0.5,-0.5]
        initial_Mu=[0.5,0.5]
        initial_observation=0
        initial_wealth=0

        ta=time.time()
        ###############    Environment

        e.reset()
        initial_observation=e.current_state
        observation=initial_observation
        state=e.current_state
        ##############       Agent
        # Create the agent
        ag=Bauerle_Rieder_agent(environment=e, num_of_Mu_chunks=2,max_iterations=max_depth)


        # Run its value iteration
        b_S,b_reachables,b_map,b_q_func,b_value_function,b_action_func=ag.continious_optimized_planning(initial_mu_state=initial_Mu,initial_observation=initial_observation,initial_wealth=0,exp_weights=exp_weights,exp_vorfaktoren=exp_vorfaktoren)
        statesused=0
        for i in range(max_depth+1):
            statesused+=len(b_reachables[i])
        print("depth:",max_depth, "states used:", statesused)
        # reset the agent 
        ag.ch_reset(initial_mu_state=initial_Mu,initial_observation=initial_observation,initial_wealth=initial_wealth)

        tb=time.time()
        print("time needed:", tb-ta)
        efficiency[i].append(tb-ta)

        #############         Simulation

        for t in range(planning_depth):

            print('t=',t)
            print('------')
            print('state:',state,e.states[state])
            print('last_observation:',observation)
            print('-------------------------------')
            print('')
            print('current internal state:',ag.current_internal_state)
            print('')

            # agent select the action
            action,value_of_action,_=ag.ch_do_action()

            #environment feedback
            t1,t2,state,reward,observation=e.step(num_to_act[action])
            total_reward+=reward

            # agent update
            new_x=ag.ch_update_agent(new_observation=observation) 

            print('             action:',num_to_act[action])  
            print ('             **' )
            print('reward:',reward,'new observation:',observation)
            print('')
            print('new internal (Mu) state:', ag.current_internal_state)
            print('')
            print('============================')


        results[i].append(total_reward)


print("results_b", results_b)
print("time_b",efficiency_b)


In [ ]:
############################## MULTIVARIATE DATA GATHERING ################################################
tx=time.time()
loops=100

results={1:[],2:[],3:[],4:[],5:[],6:[],7:[],8:[],9:[],10:[]}
comptime={1:[],2:[],3:[],4:[],5:[],6:[],7:[],8:[],9:[],10:[]}
for i in range(1,7):
    
        for j in range(loops):

            planning_depth=i
            partitioning_chunk_number=1000
            exp_vorfaktoren=[1,-1]
            exp_weights=[0.5,-0.5]
            initial_theta=[0.5,0.5]
            initial_observation=0
            modes=['discrete_optimized','naive','cheating']

            ###############    Environment
            t0 = time.time()
            e.reset()
            initial_observation=e.current_state
            observation=initial_observation
            state=e.current_state

            ##############     Agent 

            ag=Multi_Variate_agent(environment=e, planning_depth=planning_depth, partitioning_chunk_number=partitioning_chunk_number,agent_mode='cheating')
            ag.reset()
            mo_x_map,mo_M,mo_F,mo_G,mo_reachables,mo_value_function_zero,mo_all_thetas=ag.pre_planning(exp_vorfaktoren=exp_vorfaktoren,exp_weights=exp_weights, initial_theta=initial_theta,initial_observation=initial_observation,initial_wealth=0)
            mo_value_func,mo_action_func,mo_q_func,vf2=ag.value_iteration()
            t1 = time.time()
            statesused=0
            for i in range(planning_depth+1):
                statesused+=len(mo_reachables[i])
#             print("depth:",planning_depth, "states used:", statesused)
#             print("time needed:", t1-t0)
            comptime[i].append(t1-t0)

            ##############       Simulation
            total_reward=0
            for t in range(planning_depth):

#                 print('t=',t)
#                 print('------')
#                 print('state:',state,e.states[state])
#                 print('last_observation:',observation)
#                 print('-------------------------------')
#                 print('')
#                 print('current internal state:',ag.current_internal_state)
#                 print('')

                # agent select the action
                action,value_of_action=ag.do_action()

                #environment feedback
                t1,t2,state,reward,observation=e.step(num_to_act[action])
                total_reward+=reward

                # agent update
                new_x=ag.update_agent(new_observation=observation) 


#                 print('             action:',num_to_act[action])  
#                 print ('             **' )
#                 print('reward:',reward,'new observation:',observation)
#                 print('')
#                 print('new internal state:', ag.current_internal_state)
#                 print('')
#                 print('============================')
#                 print('total_reward', total_reward)

            results[i].append(total_reward)
        print(i)

print("results", results)
print("=================================")
print("time",comptime)

############################## BAUERLE DATA GATHERING ################################################


results_b={1:[],2:[],3:[],4:[],5:[],6:[],7:[],8:[],9:[],10:[]}
comptime_b={1:[],2:[],3:[],4:[],5:[],6:[],7:[],8:[],9:[],10:[]}

for i in range(1,7):
    for j in range(loops):
        total_reward=0
        max_depth=i
        planning_depth=max_depth
        exp_vorfaktoren=[1,-1]
        exp_weights=[0.5,-0.5]
        initial_Mu=[0.5,0.5]
        initial_observation=0
        initial_wealth=0

        ta=time.time()
        ###############    Environment

        e.reset()
        initial_observation=e.current_state
        observation=initial_observation
        state=e.current_state
        ##############       Agent
        # Create the agent
        ag=Bauerle_Rieder_agent(environment=e, num_of_Mu_chunks=2,max_iterations=max_depth)


        # Run its value iteration
        b_S,b_reachables,b_map,b_q_func,b_value_function,b_action_func=ag.continious_optimized_planning(initial_mu_state=initial_Mu,initial_observation=initial_observation,initial_wealth=0,exp_weights=exp_weights,exp_vorfaktoren=exp_vorfaktoren)
        statesused=0
        for i in range(max_depth+1):
            statesused+=len(b_reachables[i])
#         print("depth:",max_depth, "states used:", statesused)
        # reset the agent 
        ag.ch_reset(initial_mu_state=initial_Mu,initial_observation=initial_observation,initial_wealth=initial_wealth)

        tb=time.time()
#         print("time needed:", tb-ta)
        comptime_b[i].append(tb-ta)

        #############         Simulation

        for t in range(planning_depth):

#             print('t=',t)
#             print('------')
#             print('state:',state,e.states[state])
#             print('last_observation:',observation)
#             print('-------------------------------')
#             print('')
#             print('current internal state:',ag.current_internal_state)
#             print('')

            # agent select the action
            action,value_of_action,_=ag.ch_do_action()

            #environment feedback
            t1,t2,state,reward,observation=e.step(num_to_act[action])
            total_reward+=reward

            # agent update
            new_x=ag.ch_update_agent(new_observation=observation) 

#             print('             action:',num_to_act[action])  
#             print ('             **' )
#             print('reward:',reward,'new observation:',observation)
#             print('')
#             print('new internal (Mu) state:', ag.current_internal_state)
#             print('')
#             print('============================')


        results_b[i].append(total_reward)
    print(i)


print("results_b", results_b)
print("===============")
print("time_b",comptime_b)
ty=time.time()
print("----------------------")
print("total time needed for", loops, "loops", ty-tx, "seconds")




In [ ]:
############################## BAUERLE DATA GATHERING ################################################


results_b={1:[],2:[],3:[],4:[],5:[],6:[],7:[],8:[],9:[],10:[]}
efficiency_b={1:[],2:[],3:[],4:[],5:[],6:[],7:[],8:[],9:[],10:[]}

for i in range(1,9):
    for j in range(loops):
        total_reward=0
        max_depth=i
        planning_depth=max_depth
        exp_vorfaktoren=[1,-1]
        exp_weights=[0.5,-0.5]
        initial_Mu=[0.5,0.5]
        initial_observation=0
        initial_wealth=0

        ta=time.time()
        ###############    Environment

        e.reset()
        initial_observation=e.current_state
        observation=initial_observation
        state=e.current_state
        ##############       Agent
        # Create the agent
        ag=Bauerle_Rieder_agent(environment=e, num_of_Mu_chunks=2,max_iterations=max_depth)


        # Run its value iteration
        b_S,b_reachables,b_map,b_q_func,b_value_function,b_action_func=ag.continious_optimized_planning(initial_mu_state=initial_Mu,initial_observation=initial_observation,initial_wealth=0,exp_weights=exp_weights,exp_vorfaktoren=exp_vorfaktoren)
        statesused=0
        for i in range(max_depth+1):
            statesused+=len(b_reachables[i])
#         print("depth:",max_depth, "states used:", statesused)
        # reset the agent 
        ag.ch_reset(initial_mu_state=initial_Mu,initial_observation=initial_observation,initial_wealth=initial_wealth)

        tb=time.time()
#         print("time needed:", tb-ta)
        efficiency_b[i].append(tb-ta)

        #############         Simulation

        for t in range(planning_depth):

#             print('t=',t)
#             print('------')
#             print('state:',state,e.states[state])
#             print('last_observation:',observation)
#             print('-------------------------------')
#             print('')
#             print('current internal state:',ag.current_internal_state)
#             print('')

            # agent select the action
            action,value_of_action,_=ag.ch_do_action()

            #environment feedback
            t1,t2,state,reward,observation=e.step(num_to_act[action])
            total_reward+=reward

            # agent update
            new_x=ag.ch_update_agent(new_observation=observation) 

#             print('             action:',num_to_act[action])  
#             print ('             **' )
#             print('reward:',reward,'new observation:',observation)
#             print('')
#             print('new internal (Mu) state:', ag.current_internal_state)
#             print('')
#             print('============================')


        results_b[i].append(total_reward)



print("results_b", results_b)
print("time_b",efficiency_b)
ty=time.time()
print("===============")
#print("total time needed for", loops, "loops", ty-tx, "seconds")

